In [ ]:
import math


# Yang kurang tinggal 
# - Melakukan perubahan pada data sehingga value untuk setiap atribut adalah numerik
# - Contohnya value pada jenis kelamin sebelumnya male, female diganti menjadi 0,1 atau terserah
# - Sehingga nanti value setiap atribut tidak ada yang string => sudah numerik semua

# Dataset sementara untuk test algoritma
# Dataset dari slide yang untuk K-Medoids
# Dataset memiliki ukuran 10
# Terdiri dari dua atribut untuk setiap dataset

dataset = []
dataset.append([2,6])
dataset.append([3,4])
dataset.append([3,8])
dataset.append([4,7])
dataset.append([6,2])
dataset.append([6,4])
dataset.append([7,3])
dataset.append([7,4])
dataset.append([8,5])
dataset.append([7,6])

# cluster[0] maksudnya cluster dengan banyak cluster = 10 => masih belum ada data yang digabung
# cluster[1] maksudnya cluster dengan banyak cluster = 9 => sudah ada dua data yang digabung menjadi satu cluster
# cluster[9] maksudnya cluster dengan banyak cluster = 1 => semua data sudah berada pada satu cluster
# ...
# cluster[n][1] adalah menyatakan daftar indeks data yang termasuk dalam satu cluster 

cluster = {}

for i in range(len(dataset)):
    cluster[i] = []

for i in range(len(dataset)):
    temp = []
    temp.append(i)
    cluster[0].append(temp)
    
print(cluster)


In [ ]:
# Menghitung jarak data antara data 1 terhadap data 2
# Jarak yang digunakan adalah jarak euclidean

def hitungjarakantardata(data1,data2):
    jarakkuadrat = 0
    for i in range(len(data1)):
        jarakkuadrat += pow(data1[i]-data2[i],2)
    return math.sqrt(jarakkuadrat)

# Matrix jarak antar semua data di dataset
def matrixjarak(dataset):
    array = [[0 for x in range(len(dataset))] for y in range(len(dataset))] 
    
    for i in range(len(dataset)):
        for j in range(i+1,len(dataset)):
            array[i][j] = hitungjarakantardata(dataset[i],dataset[j])
    
    return array

# Menghitung jarak antar cluster
# Hitung jarak clusternya masih menggunakan single link
def hitungjarakantarcluster(cluster1,cluster2):
    jarakterdekat = 999
    for i in range(len(cluster1)):
        for j in range(len(cluster2)):
            if(hitungjarakantardata(dataset[cluster1[i]],dataset[cluster2[j]]) < jarakterdekat):
                jarakterdekat = hitungjarakantardata(dataset[cluster1[i]],dataset[cluster2[j]])
    return jarakterdekat

# Mencari cluster yang memiliki jarak terdekat 
def cariclusterterdekat(alcluster):
    jarakterdekat = 999
    for i in range(len(alcluster)):
        for j in range(i+1,len(alcluster)):
            if(hitungjarakantarcluster(alcluster[i],alcluster[j]) < jarakterdekat):
                jarakterdekat = hitungjarakantarcluster(alcluster[i],alcluster[j])
                idx = []
                idx.append(i)
                idx.append(j)
    return idx

In [ ]:
print("Cluster di 0",cluster[0])

# Reset seluruh cluster[1] sampai cluster [N-1] menjadi new
for i in range(1,len(cluster)):
    cluster[i] = []

# Algoritma Utama Agglomerative

for i in range(1,len(cluster)):
    arrindex = cariclusterterdekat(cluster[i-1])
    indeks1 = arrindex[0]
    indeks2 = arrindex[1]
    temp = []
    for j in range(len(cluster[i-1][indeks1])):
        temp.append(cluster[i-1][indeks1][j])
        
    for k in range(len(cluster[i-1][indeks2])):
        temp.append(cluster[i-1][indeks2][k])
    cluster[i].append(temp)
    #print("Sebelum",i,cluster[i])
    
    for l in range(len(cluster[i-1])):
        if((l != indeks1) & (l != indeks2)):
            cluster[i].append(cluster[i-1][l])
    print("Cluster di",i,cluster[i])